In [ ]:
from keras.models import load_model
!pip install mtcnn
import mtcnn
from PIL import Image
from numpy import asarray
from mtcnn.mtcnn import MTCNN
from os import listdir
from matplotlib import pyplot
from os.path import isdir
from numpy import savez_compressed
from numpy import load
from numpy import expand_dims
# Load the model
model=load_model('/facenet_keras.h5')
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
import matplotlib
import matplotlib.patches as patches


In [ ]:
# Extract face pixels from a given image.
def faceExt(imgpath, size=(160,160)):
  img=Image.open(imgpath)
  img=img.convert('RGB')
  pix=asarray(img)
  detector=MTCNN()
  results=detector.detect_faces(pix)
  x1,y1,wd,ht=results[0]['box']
  x1,y1=abs(x1),abs(y1)
  x2,y2=x1+wd,y1+ht
  #print('ordinates')
  #print(x1,y1,x2,y2) 
  face=pix[y1:y2,x1:x2]
  img=Image.fromarray(face)
  img=img.resize(size)
  farray=asarray(img)
  return farray

In [ ]:
# Load faces from images stored in the given directory.
def load_faces(dir):
  faces=[]  
  for file in listdir(dir):
    path=dir+'/'+file`
    try:
      face=faceExt(path)
    except Exception as er:
      print(file)
    faces.append(face)
  return faces

In [ ]:
# Provide the path for the dataset directory to extract faces from the dataset using the aforementioned functions.
def load_dataset(directory):
  x=[]
  y=[]
  #print(listdir(directory))
  for subdir in listdir(directory):
    #print(subdir)
    path = directory + subdir 
    #print(path)
    if not isdir(path):
      continue
    try:
      faces = load_faces(path)
    except Exception as er:
      print(er)
    labels = [subdir for _ in range(len(faces))]
    print('>loaded %d examples for class: %s' % (len(faces), subdir))
    x.extend(faces)
    y.extend(labels)
  return asarray(x),asarray(y)

In [ ]:
trainX, trainy = load_dataset('/train/')# path for train dataset
print(trainX.shape, trainy.shape)
testX, testy = load_dataset('/val/')# path for test dataset
savez_compressed('faces_dataset.npz', trainX, trainy, testX, testy)# Save the extracted faces from the dataset as a compressed file.


In [ ]:
data = load('faces_dataset.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Loaded: ', trainX.shape, trainy.shape, testX.shape, testy.shape)

In [ ]:
# Extraxt face embeddings by traversing the face pixels through the FaceNet model.
def get_embedding(model,face):
  face=face.astype('float32')
  mean=face.mean()
  std=face.std()
  face=(face-mean)/std
  sample=expand_dims(face,axis=0)
  embedding=model.predict(sample)
  return embedding[0]

In [ ]:
emd_train=[]
for face in trainX:
  embedding=get_embedding(model,face)
  emd_train.append(embedding)
emd_train=asarray(emd_train)
emd_test=[]
for face in testX:
  embedding=get_embedding(model,face)
  emd_test.append(embedding)
emd_test=asarray(emd_test)
savez_compressed('face_embeddings.npz',emd_train,trainy,emd_test,testy)# save embeddings as a compressed file.

In [ ]:
# Train the SVM model based on the face embeddings.
data = load('face_embeddings.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
incoder=Normalizer(norm='l2')
trainX=incoder.transform(trainX)
testX=incoder.transform(testX)
outcoder=LabelEncoder()
outcoder.fit(trainy)
trainy=outcoder.transform(trainy)
testy=outcoder.transform(testy)
model=SVC(kernel='linear',probability=True)
model.fit(trainX,trainy)
yhat_train = model.predict(trainX)
yhat_test = model.predict(testX)
score_train = accuracy_score(trainy, yhat_train)
score_test = accuracy_score(testy, yhat_test)
print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))

In [ ]:
# Function to draw face contours with the identity of the person.
def draw(filename,name):
  im=Image.open(filename)
  im=im.convert('RGB')
  pix=asarray(im)
  detector=MTCNN()
  results=detector.detect_faces(pix)  
  x1,y1,wd,ht=results[0]['box']
  x1,y1=abs(x1),abs(y1)
  img = matplotlib.image.imread(filename)
  figure, ax = pyplot.subplots(1)
  rect = patches.Rectangle((x1,y1),wd,ht, edgecolor='r', facecolor="none")
  ax.imshow(im)
  ax.add_patch(rect)
  ax.text(x1,y1,name,style ='italic', fontsize = 12,color='black', bbox ={'facecolor':'white','alpha':0.6, 'pad':5})

In [ ]:
def recognize(path):
  face_pixels=faceExt(path)
  face_embedding=get_embedding(load_model('/facenet_keras.h5'),face_pixels)
  samples=expand_dims(face_embedding,axis=0)
  predicted_class=model.predict(samples)
  predicted_probability=model.predict_proba(samples)
  if predicted_probability[0,predicted_class[0]]>0.65:
    name=outcoder.inverse_transform(predicted_class)
    draw(path,name[0])
  else:
    draw(path,'unknown person')

In [ ]:
recognize('train/ben_afflek/httpcsvkmeuaeccjpg.jpg')